# PyTorch - Night to Day
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/PyTorch/PyTorch_Night_to_Day.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

### Imports

In [1]:
import torch
from torch import nn
import PIL
import requests
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms as T
from model import Generator_Resnet

In [2]:
transform = T.Compose([
        T.ToTensor(),
        T.Resize((300, 300))
    ])

### INPUT : Image of Dimension 300 x 300

In [3]:
img_url = "https://lh3.googleusercontent.com/Mjz79x3QRUldbe6VJhfw8xxWcvnZ9YT0P3KJxT1KdUzolrovEMxuUjoti8Xtk3uDq5ntWr_04C2KtU6wqg3b=w2880-h1642"
in_img = Image.open(requests.get(img_url, stream=True).raw) # change the path to the input image
in_img = torch.reshape(transform(in_img), (1, 3, 300, 300))

In [4]:
_, ax = plt.subplots(figsize=(5, 5))
ax.axis("off")
ax.imshow(np.array(in_img).reshape(3, 300, 300).transpose(1, 2, 0), aspect="auto", interpolation="nearest")

### Model : Cycle Generative Adversarial Networks ( GAN ) with MSE and L1 loss

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        # print(in_features)

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.LeakyReLU(inplace=True),
            nn.ReflectionPad2d(2),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.Conv2d(in_features, in_features, 3)
        )

    def forward(self, x):
        out = self.block(x)
        return x + out


class Generator_Resnet(nn.Module):
    def __init__(self, in_shape, n_res_blocks=5):
        super(Generator_Resnet, self).__init__()
        channels = in_shape[0]
        n_blocks = n_res_blocks

        out_features = 64

        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.LeakyReLU(inplace=True)
        ]
        in_features = out_features

        # Downscaling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.LeakyReLU(inplace=True)
            ]
            in_features = out_features

        model += [ResidualBlock(in_features) for _ in range(n_blocks)]

        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

            # Output layer
        # print(out_features)
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

In [6]:
Gen_Night2Day = Generator_Resnet((3, 300, 300), 4)
Gen_Night2Day.load_state_dict(torch.load("./G_AB.pth", map_location=torch.device("cpu")))

### Output : Image of Dimension 300 x 300

In [7]:
out_img = Gen_Night2Day(in_img).detach().numpy().reshape((3, 300, 300)).transpose(1, 2, 0)

In [8]:
_, ax = plt.subplots(1, 2, figsize=(15, 8))
ax[0].axis('off')
ax[1].axis('off')
ax[0].set_title("Input Night image", fontsize=20)
ax[1].set_title("Output Day image", fontsize=20)
ax[0].imshow(np.array(in_img).reshape(3, 300, 300).transpose(1, 2, 0), aspect="auto", interpolation="nearest")
ax[1].imshow(out_img, interpolation="nearest", aspect="auto")